## Disclaimer
This repository is a conceptual research prototype. References to "AGI" and "superintelligence" describe aspirational goals and do not indicate the presence of a real general intelligence. Use at your own risk.

# 🧬 AI-GA Meta-Evolution · Colab

Fully-featured notebook that spins up the **three-pillar** demo from Clune (2020) in ~2 minutes.

**What you get**
1. 📦 Minimal runtime install (CPU ⬅️ default, GPU optional)
2. 🔑 Optional `OPENAI_API_KEY` for LLM commentary (offline Mixtral otherwise)
3. 🚀 `agent_aiga_entrypoint.py` exposing:
   * Gradio dashboard → port `7862` (public HTTPS link auto-printed)
   * FastAPI JSON API → port `8000` (see `/docs`)
4. 🧪 Unit-test cell (`pytest -q`) – target **≥ 90 % branch coverage** < 0.5 s
5. 📈 Inline Prometheus scrape & Matplotlib plot
6. 🤖 Example REST calls (evolve / checkpoint) using `httpx`

In [ ]:
#@title ↳ Choose runtime {display-mode: "form"}
USE_GPU = False  #@param {type:"boolean"}
print("⚙️  GPU runtime" if USE_GPU else "⚙️  CPU runtime")

In [ ]:
# ↳ Setup (≈90 s CPU / 60 s GPU) -----------------------------------------
%%bash --no-stderr
set -Eeuo pipefail

# clone
if [[ ! -d AGI-Alpha-Agent-v0 ]]; then
  echo "📡  Cloning Alpha-Factory repo …" >&2
  git clone --depth 1 https://github.com/MontrealAI/AGI-Alpha-Agent-v0.git -q
fi
cd AGI-Alpha-Agent-v0/alpha_factory_v1/demos/aiga_meta_evolution

# wheel index depending on GPU flag
WHL_URL="https://download.pytorch.org/whl/$(python - <<PY
import os; print('cu118' if os.environ.get('USE_GPU','False')=='True' else 'cpu')
PY)"

echo "📦  Installing deps …" >&2
pip -q install --upgrade pip
pip -q install torch torchvision --extra-index-url $WHL_URL
pip -q install gymnasium[classic_control] gradio==4.* openai_agents httpx prometheus-client pytest coverage scipy


### ⬆️ Upload a wheelhouse (optional)
Use the file browser on the left to upload a `wheels.zip` archive. Unzip it
and point `check_env.py` at the extracted directory:
```bash
WHEELHOUSE=/content/wheels
unzip -q wheels.zip -d "$WHEELHOUSE"
python AGI-Alpha-Agent-v0/check_env.py --auto-install --wheelhouse "$WHEELHOUSE"
```


In [ ]:
# ↳ (optional) supply OpenAI key ------------------------------------------
import os, getpass

if not os.getenv('OPENAI_API_KEY'):
    key = getpass.getpass('Paste OPENAI_API_KEY, or press Enter for offline mode: ')
    if key:
        %env OPENAI_API_KEY=$key
        print('🔑  Key set – online mode.')
    else:
        print('🛰  Offline mode – Mixtral fallback.')


In [ ]:
# ↳ Quick unit tests ------------------------------------------------------
%%bash
cd AGI-Alpha-Agent-v0/alpha_factory_v1/demos/aiga_meta_evolution
pytest -q || echo '⚠️ tests failed (safe to ignore for demo)'


In [ ]:
# ↳ Launch the orchestration service & dashboard --------------------------
import subprocess, threading, re, time, sys, pathlib, os, json

ROOT = pathlib.Path("AGI-Alpha-Agent-v0/alpha_factory_v1/demos/aiga_meta_evolution").resolve()
os.chdir(ROOT)

proc = subprocess.Popen([sys.executable, "agent_aiga_entrypoint.py"],
                        stdout=subprocess.PIPE, stderr=subprocess.STDOUT,
                        text=True)

public = None
def _tail():
    global public
    for line in proc.stdout:
        print(line, end="")
        if not public and "Running on" in line and "https" in line:
            public = re.search(r"https?://[\w./-]+", line)[0]
            print(f"\n🔗  Dashboard → {public}\n")
threading.Thread(target=_tail, daemon=True).start()

for _ in range(90):
    if public: break
    time.sleep(1)
if not public:
    print("⏳  Still starting … open logs above for status.")

## ☎️ API interactions
Below we hit the running FastAPI service using `httpx`.


In [ ]:
import httpx, time, pandas as pd, matplotlib.pyplot as plt

API = "http://localhost:8000"
print(httpx.get(API + "/health").json())

# schedule 20 generations asynchronously
httpx.post(API + "/evolve/20").json()

In [ ]:
# ↳ scrape Prometheus metrics & plot --------------------------------------
import re, json, pandas as pd, matplotlib.pyplot as plt, httpx, time

raw = httpx.get(API + "/metrics").text
fitness = float(re.search(r"aiga_best_fitness (\d+\.?\d*)", raw).group(1))
gen     = int(re.search(r"aiga_generations_total (\d+)", raw).group(1))
print(f"Generation {gen}, best fitness {fitness:.2f}")

## 📊 Plot fitness history
The checkpoint JSON embeds the `(generation, avg_fitness)` list.

In [ ]:
import json, pathlib, matplotlib.pyplot as plt, pandas as pd, glob
ckpt = sorted(glob.glob("checkpoints/evolver_gen*.json"))[-1]
hist = json.loads(pathlib.Path(ckpt).read_text())["history"]
df = pd.DataFrame(hist, columns=["gen", "avg"])
df.plot(x="gen", y="avg", figsize=(6,3), grid=True, legend=False)

---
## 🏗 Running on Kubernetes (FYI)
```yaml
apiVersion: apps/v1
kind: Deployment
metadata: { name: aiga-demo }
spec:
  replicas: 1
  selector: { matchLabels: { app: aiga-demo } }
  template:
    metadata: { labels: { app: aiga-demo } }
    spec:
      containers:
      - name: orchestrator
        image: ghcr.io/montrealai/alpha-aiga:latest@sha256:<signed>
        envFrom:
        - secretRef: { name: aiga-secrets }
        ports:
        - containerPort: 8000
        - containerPort: 7862
```
Full SOC-2 artefacts (audit logs, SBOM, cosign signature) ship with the container.

### 🎯 Next steps
* Increase `pop_size` or enable GPU for faster evolution.
* Modify `curriculum_env.py` to add novel stages.
* Plug the JSON API into your own micro-services for autonomous decision loops.

In [ ]:
# ↳ Graceful shutdown ----------------------------------------------
proc.terminate()
proc.wait(timeout=10)
print('✅  Service stopped.')
